<a href="https://colab.research.google.com/github/dbamman/nlp23/blob/main/HW1/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1: Featurized Models for Sentiment Analysis

For this assignment, we provide an implementation of a simple binary classifier that will predict the sentiment of a movie review based on a group of original features -- provided by you! 

Before diving into any code, please read through the associated [PDF](https://github.com/dbamman/nlp23/blob/main/HW1/HW1.pdf) for an overview of the assignment and specific instructions on how to submit.

In [1]:
import sys, argparse
from scipy import sparse
from sklearn import linear_model
from collections import Counter
import numpy as np
import re
from collections import Counter, defaultdict
import operator
import nltk
import csv
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
from pandas import option_context


In [2]:
# !python -m nltk.downloader punkt

### Intro: Gather Data + Create Classifier

#### Gather Data

Let's download the data we'll use for training and development, and also the data we'll make predictions with.

In [3]:
### Commenting out since working locally
#  # Get data
# !wget https://raw.githubusercontent.com/dbamman/nlp23/main/HW1/train.txt
# !wget https://raw.githubusercontent.com/dbamman/nlp23/main/HW1/dev.txt
# !wget https://raw.githubusercontent.com/dbamman/nlp23/main/HW1/test.txt

In [4]:
trainingFile = "train.txt"
evaluationFile = "dev.txt"
testFile = "test.txt"

lexiconFile = "subjclueslen1-HLTEMNLP05.tff"

#### Define Classifier class

Next, we've created a Binary Classifier. This class will let us learn the traits associated with positive and negatively classed movie reviews in order to make predictions on our test set.

In [42]:
######################################################################
## Do not edit this block of code.
## This defines the classification class which
## loads the data and sets up the model.
######################################################################

class Classifier:

    def __init__(self, feature_method, L2_regularization_strength=1.0, min_feature_count=1):
        self.feature_vocab = {}
        self.feature_method = feature_method
        self.log_reg = None
        self.L2_regularization_strength=L2_regularization_strength
        self.min_feature_count=min_feature_count

        self.trainX, self.trainY, self.trainOrig = self.process(trainingFile, training=True)
        self.devX, self.devY, self.devOrig = self.process(evaluationFile, training=False)
        self.testX, _, self.testOrig = self.process(testFile, training=False)

    # Read data from file
    def load_data(self, filename):
        data = []
        with open(filename, encoding="utf8") as file:
            for line in file:
                cols = line.split("\t")
                idd = cols[0]
                label = cols[1]
                text = cols[2]

                data.append((idd, label, text))
                
        return data

    # Featurize entire dataset
    def featurize(self, data):
        featurized_data = []
        for idd, label, text in data:
            feats = self.feature_method(text)
            featurized_data.append((label, feats))
        return featurized_data

    # Read dataset and returned featurized representation as sparse matrix + label array
    def process(self, dataFile, training = False):
        original_data = self.load_data(dataFile)
        data = self.featurize(original_data)

        if training:
            fid = 0
            feature_doc_count = Counter()
            for label, feats in data:
                for feat in feats:
                    feature_doc_count[feat]+= 1

            for feat in feature_doc_count:
                if feature_doc_count[feat] >= self.min_feature_count:
                    self.feature_vocab[feat] = fid
                    fid += 1

        F = len(self.feature_vocab)
        D = len(data)
        X = sparse.dok_matrix((D, F))
        Y = [None]*D
        for idx, (label, feats) in enumerate(data):
            for feat in feats:
                if feat in self.feature_vocab:
                    X[idx, self.feature_vocab[feat]] = feats[feat]
            Y[idx] = label

        return X, Y, original_data

    def load_test(self, dataFile):
        data = self.load_data(dataFile)
        data = self.featurize(data)

        F = len(self.feature_vocab)
        D = len(data)
        X = sparse.dok_matrix((D, F))
        Y = [None]*D
        for idx, (data_id, feats) in enumerate(data):
            for feat in feats:
                if feat in self.feature_vocab:
                    X[idx, self.feature_vocab[feat]] = feats[feat]
            Y[idx] = data_id

        return X, Y

    # Train model and evaluate on held-out data
    def evaluate(self):
        (D,F) = self.trainX.shape
        self.log_reg = linear_model.LogisticRegression(C = self.L2_regularization_strength, max_iter=1000)
        self.log_reg.fit(self.trainX, self.trainY)
        training_accuracy = self.log_reg.score(self.trainX, self.trainY)
        development_accuracy = self.log_reg.score(self.devX, self.devY)
        print("Method: %s, Features: %s, Train accuracy: %.3f, Dev accuracy: %.3f" % (self.feature_method.__name__, F, training_accuracy, development_accuracy))


    # Predict labels for new data
    def predict(self):
        predX = self.log_reg.predict(self.testX)

        with open("%s_%s" % (self.feature_method.__name__, "predictions.csv"), "w", encoding="utf8") as out:
            writer=csv.writer(out)
            writer.writerow(["Id", "Expected"])
            for idx, data_id in enumerate(self.testX):
                writer.writerow([self.testOrig[idx][0], predX[idx]])
        out.close()


    def printWeights(self, n=10):

        reverse_vocab=[None]*len(self.log_reg.coef_[0])
        for k in self.feature_vocab:
            reverse_vocab[self.feature_vocab[k]]=k

        # binary
        if len(self.log_reg.classes_) == 2:
              weights=self.log_reg.coef_[0]

              cat=self.log_reg.classes_[1]
              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

              cat=self.log_reg.classes_[0]
              for feature, weight in list(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1)))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

        # multiclass
        else:
          for i, cat in enumerate(self.log_reg.classes_):

              weights=self.log_reg.coef_[i]

              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

            

#### Simple Classifier example

Let's create an initial classifier based on a really simple feature using a dictionary: 

* if the abstract contains the words "love" or "like", the `contains_positive_word` feature will fire, and 
* if it contains either "hate" or "dislike", the `contains_negative_word` will fire.  

Note how we use `nltk.word_tokenize` to tokenize the text into its discrete words (the documentation for which can be found [here](https://www.nltk.org/api/nltk.tokenize.html)).

In [6]:
def simple_featurize(text):
    feats = {}
    words = nltk.word_tokenize(text)

    for word in words:
        word=word.lower()
        if word == "love" or word == "like":
            feats["contains_positive_word"] = 1
        if word == "hate" or word == "dislike":
            feats["contains_negative_word"] = 1
            
    return feats


Now let's see how that feature performs on the development data. 

Note the `L2_regularization_strength` parameter specifies the strength of the L2 regularizer (values closer to 0 = stronger regularization), and `min_feature_count` specifies how many data points need to contain a feature for it to be passed into the model as a feature. Both are ways to prevent the model from overfitting and achieve higher performance.

In [7]:
simple_classifier = Classifier(simple_featurize, L2_regularization_strength=1.0, min_feature_count=1)
simple_classifier.evaluate()

Method: simple_featurize, Features: 2, Train accuracy: 0.509, Dev accuracy: 0.500


So we've created a classifier. But is its accuracy score any good?  Let's calculate the accuracy of a "majority classifier" to provide some context. This determines the most-represented (majority) class in the training data, and then predicts every test point to be this class.

In [8]:
def majority_class(trainY, devY):
    labelCounts=Counter()
    for label in trainY:
        labelCounts[label]+=1
    majority_class=labelCounts.most_common(1)[0][0]

    correct=0.
    for label in devY:
        if label == majority_class:
            correct+=1
            
    print("Majority class: %s\tDev accuracy: %.3f" % (majority_class, correct/len(devY)))
majority_class(simple_classifier.trainY, simple_classifier.devY)

Majority class: pos	Dev accuracy: 0.500


The feature we created in `simple_featurize`, evidently, doesn't have a whole lot of legs. In the next portion of the homework, you'll be designing a few features of your own in the hopes of achieving the highest accuracy possible.

## Deliverable 1

Your job in this homework is to implement a binary bag-of-words model (i.e., one that assigns a feature value of 1 to each word type that is present in the text); and to brainstorm **3 additional** distinct classes of features, justify why they might help improve the performance *over a bag of words* for this task, implement them in code, and then assess their independent performance on the development data.

To show your work: describe your new features and report their performance in the table below; implement the features in the specified `feature1`, `feature2`, and `feature3` functions, and execute each respective classifier to show its performance.  

|Feature|Why should it work? (50 words each)|Dev set performance|
|---|---|---|
|Bag of words| Captures much of the variance between texts by using presence of individual tokens. | 0.776
|Feature 1: Text length and uniqueness |  Critics may write different length reviews for positive vs. negative movies, as well as use a more varied vocabulary. For this set of features I tokenize the text, remove punctuation, then take the length of the tokenized text. Then I applied the natural log to reduce skew of outlier length texts. I also took the proportion of unique tokens to the text length. This captures a different dimension of the text than the individual tokens by treating the text as a whole. | 0.508
|Feature 2: Bigrams | The bigrams help encode some context from the original text in the features. TO implement this, I used NLTK's bigrams method to create a dictionary where there was a binary indicator for each bigram present. | 0.763
|Feature 3: MPQA Subjectivity Lexicon | I leveraged the subjectivity lexicon from MPQA to assign an overall + or - binary label to each text, depending on the majority count of the lexicon's highly subjective words in the text. I expect this to add an additional dimension of sentiment to the text that the bag of words doesn't necessarily capture. (note this is labeled as feature5 below) | 0.659

Note that it is not required for your features to actually perform well, but your justification for why it *should* perform better than a bag of words should be defensible.  The most creative features (defined as features that few other students use and that are reasonably well-performing) will receive extra credit for this assignment. Consider the type of data you are working with: what do you look for when writing/reading a movie review?

### Implement Bag-of-Words

In [71]:
def bag_of_words(text):
    # Here the `feats` dict should contain the features -- the key should be the feature name, 
    # and the value is the feature value.  See `simple_featurize` for an example.
    
    # BEGIN SOLUTION
    words = [x.lower() for x in nltk.word_tokenize(text)]
    words_set = list(set(words))
    words_values = np.ones(len(words_set))
    feats = dict(zip(words_set, words_values))

    return feats
    # END SOLUTION

In [72]:
######################################################################
## Do not edit this block of code, except for the L2_regularization_strength and min_feature_count parameters
######################################################################

bow_classifier = Classifier(bag_of_words, L2_regularization_strength=1.0, min_feature_count=1)
bow_classifier.evaluate()

KeyboardInterrupt: 

### Implement Original Features

In [132]:
def feature1(text):
    # Here the `feats` dict should contain the features -- the key should be the feature name, 
    # and the value is the feature value.  See `simple_featurize` for an example.
    ''''Implements review length feature'''
    feats = {}
    words = [x.lower() for x in nltk.word_tokenize(text) if not re.search('[.,!?:;]', x)]
    words_set = len(set(words))
    prop = words_set / len(words)
    feats["uniqueness"] = prop
    feats["length"] = np.log(len(words))
               
    return feats

In [133]:
######################################################################
## Do not edit this block of code, except for the L2_regularization_strength and min_feature_count parameters
######################################################################

classifier1 = Classifier(feature1, L2_regularization_strength=1.0, min_feature_count=1)
classifier1.evaluate()

Method: feature1, Features: 2, Train accuracy: 0.51700, Dev accuracy: 0.50800


In [136]:
def feature2(text):
    # Here the `feats` dict should contain the features -- the key should be the feature name, 
    # and the value is the feature value.  See `simple_featurize` for an example.
    '''Implements bigram feature'''
    # tokenize text --> list in order of words
    words = [x.lower() for x in nltk.word_tokenize(text)]
    # split tokens into bigrams
    bigrams = list(nltk.bigrams(words))

    # Counter object to count unique pairs and store to dict
    feats = {k: 1 for k,v in Counter(bigrams).items()}
    # print(feats)
    # feats = dict(Counter(bigrams))
            
    return feats

In [137]:
######################################################################
## Do not edit this block of code, except for the L2_regularization_strength and min_feature_count parameters
######################################################################

classifier2 = Classifier(feature2, L2_regularization_strength=1.0, min_feature_count=1)
classifier2.evaluate()

Method: feature2, Features: 122886, Train accuracy: 1.00000, Dev accuracy: 0.76300


In [61]:
# def feature3(text):
#     # Here the `feats` dict should contain the features -- the key should be the feature name, 
#     # and the value is the feature value.  See `simple_featurize` for an example.
#     '''Implements capital letter feature'''
#     feats = {}
#     words = nltk.word_tokenize(text)
#     counts = 0
#     for w in words:
#         if (w == w.upper()) and (w != "I"):
#             counts += 1
#     feats['upper'] = np.log(counts)
            
#     return feats

In [62]:
# ######################################################################
# ## Do not edit this block of code, except for the L2_regularization_strength and min_feature_count parameters
# ######################################################################

# classifier3 = Classifier(feature3, L2_regularization_strength=1.0, min_feature_count=1)
# classifier3.evaluate()

Method: feature3, Features: 1, Train accuracy: 0.52400, Dev accuracy: 0.52700


In [68]:
# Set up Lexicon
rawSubjLexicon = []
with open(lexiconFile) as f:
    for l in f:
        rawSubjLexicon.append(l)

subjLexicon = {}
for x in rawSubjLexicon:
    d = dict(re.findall(r'(\w+)=(\S+)', x))
    # filter out strongsubj and len=1 s
    if d['len'] == '1' and d['type'] == 'strongsubj':
        subjLexicon[d['word1']] = d['priorpolarity']


In [69]:
def feature5(text):
    # Here the `feats` dict should contain the features -- the key should be the feature name, 
    # and the value is the feature value.  See `simple_featurize` for an example.
    '''Implements subjectivity lexicon features'''
    feats = {}
    words = [x.lower() for x in nltk.word_tokenize(text)]
    subjScore = 0
     
    for w in words:
        # look up in lexicon dict
        if w in subjLexicon.keys(): 
            # if positive, add 1 to subjScore
            if subjLexicon[w] == 'positive':
                subjScore += 1
            # if negative, subtract 1 from subjScore
            elif subjLexicon[w] == 'negative':
                subjScore -= 1
        else:
            continue
    # if overall subjScore is + then overall_positive = 1
    # if overall subjScore is -, then overall_negative = 1
    
    if subjScore > 0:
        feats["overallPositive"] = 1
    elif subjScore < 0:
        feats["overallNegative"] = 1

    return feats

In [70]:
######################################################################
## Do not edit this block of code, except for the L2_regularization_strength and min_feature_count parameters
######################################################################

classifier5 = Classifier(feature5, L2_regularization_strength=1.0, min_feature_count=1)
classifier5.evaluate()

Method: feature5, Features: 2, Train accuracy: 0.66300, Dev accuracy: 0.65900


## Deliverable 2

The two cells in "Combine your features" will generate a file named `combiner_function_predictions.csv`.

Download this file (using e.g. the file manager on the left panel in Colab) and submit this to GradeScope along with your notebook; the 5 students with the highest performance (revealed after the submission deadline) will receive extra credit for this assignment.

Please do not change the auto-generated filename!

### Combine your features

Next, let's combine any or all the features you have developed into one big model and make predictions on the test data. There is no exact number/threshold we're looking for, accuracy-wise, but the combiner function should *generally* have a higher accuracy than BoW on its own (assuming your features are adding additional information beyond what BoW is adding). 

You don't need to edit the following cell, unless you want to change which features are handed off to the "combined" model.

In [63]:
def combiner_function(text):

    # Here the `all_feats` dict should contain the features -- the key should be the feature name, 
    # and the value is the feature value.  See `simple_featurize` for an example.
    # at the moment, all 4 of: bag of words and your 3 original features are handed off to the combined model
    # update the values within [bag_of_words, feature1, feature2, feature3] to change this.
    
    all_feats={}
    for feature in [bag_of_words, feature1, feature2, feature5]:
        all_feats.update(feature(text))
    return all_feats

In [64]:
######################################################################
## Do not edit this block of code, except for the L2_regularization_strength and min_feature_count parameters
######################################################################


big_classifier = Classifier(combiner_function, L2_regularization_strength=0.6, min_feature_count=1)
big_classifier.evaluate()

# generate .csv file with prediction output on test data
big_classifier.predict()

Method: combiner_function, Features: 148962, Train accuracy: 1.00000, Dev accuracy: 0.81300


KeyboardInterrupt: 

## Further Exploration: Interrogating classifiers

Note: No deliverables are in this section; it's optional. Treat this portion as a useful tool for further understanding the features you worked on and how they affter the classifier.

Below you will find several ways in which you can interrogate your model to get ideas on ways to improve its performance.  

1. First, let's look at the confusion matrix of its predictions (where we can compare the true labels with the predicted labels). What kinds of mistakes is it making? (While this is mainly helpful in the context of multiclass classification, we can still see if there's a bias toward predicting a specific class in the binary setting as well). 

In [ ]:
def print_confusion(classifier):
    fig, ax = plt.subplots(figsize=(10,10))
    plot_confusion_matrix(classifier.log_reg, classifier.devX, classifier.devY, ax=ax, xticks_rotation="vertical", values_format="d")
    plt.show()

print_confusion(big_classifier)

2. Next, let's look at the features that are most important for each of the classes (ranked by how strong their corresponding coefficient is). Do the features you are defining help in the ways you think they should?  Do sets of successful features suggests others, or complementary features that may provide a different view on the data?

In [ ]:
big_classifier.printWeights()


3. Next, let's look at the individual data points that are the hardest to classify correctly. Does it suggest any features you might create to disentangle them?

In [ ]:
def analyze(classifier):
    
    probs=classifier.log_reg.predict_proba(classifier.devX)
    predicts=classifier.log_reg.predict(classifier.devX)

    classes={}
    for idx, lab in enumerate(classifier.log_reg.classes_):
        classes[lab]=idx

    mistakes={}
    for i in range(len(probs)):
        if predicts[i] != classifier.devY[i]:
            predicted_lab_idx=classes[predicts[i]]
            mistakes[i]=probs[i][predicted_lab_idx]

    frame=[]
    sorted_x = sorted(mistakes.items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_x:
        idd=classifier.devOrig[k][0]
        text=classifier.devOrig[k][2]
        frame.append([idd, v, classifier.devY[k], predicts[k], text])

    df=pd.DataFrame(frame, columns=["id", "P(predicted class confidence)", "Human label", "Prediction", "Text"]).sort_values(by='Human label')

    with option_context('display.max_colwidth', 400):
        display(df.head(n=20))


In [ ]:
analyze(big_classifier)